## Enriquecimiento del Dataset de Steam Games

El dataset original **steam_games.json**, descargado de la fuente [UCSD Datasets](https://cseweb.ucsd.edu/~jmcauley/datasets.html), no incluye la **descripción de los juegos**, información necesaria para generar *embeddings* en el sistema de recomendación.

Para enriquecer el dataset y añadir el campo **descripción**, se utilizará la API oficial de Steam: https://store.steampowered.com/api/appdetails


De esta manera, cada juego contará con su respectiva descripción, lo que permitirá mejorar la calidad del análisis y generacion de recomendaciones.


In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import os
import time
from ast import literal_eval

In [ ]:
ARCHIVO_GAMES = 'steam_games.json'
ARCHIVO_SALIDA = 'steam_games_enriched.csv'
GUARDAR_CADA_N = 50

## CARGA DE DATOS PRINCIPAL

In [ ]:
def load_json_chunks(filepath):
    """
    Función auxiliar para leer archivos JSON grandes línea por línea (formato UCSD)
    """
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for _, line in enumerate(f):
            data.append(literal_eval(line))

    return pd.DataFrame(data)

In [ ]:
df_games = load_json_chunks('steam_games.json')

In [ ]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN



## ENRIQUECIMIENTO CON STEAM API


In [ ]:
def get_steam_game_details(app_id):
    if not app_id: return None
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            data = response.json()
            if data and str(app_id) in data and data[str(app_id)]['success']:
                game_data = data[str(app_id)]['data']
                return {
                    'detailed_description': game_data.get('detailed_description', ''),
                    'short_description': game_data.get('short_description', ''),
                    'about_the_game': game_data.get('about_the_game', '')
                }
        elif response.status_code == 429:
            print("Rate limit API Steam. Esperando 5s...")
            time.sleep(5)
    except Exception as e:
        pass
    return None

In [ ]:
# Función para eliminar caracteres ilegales
import re

def limpiar_texto(texto):
    if pd.isna(texto):
        return texto
    # Elimina caracteres de control ASCII (0–31)
    return re.sub(r'[\x00-\x1F]', '', str(texto))

In [ ]:
# Asegurar ID string
if 'id' in df_games.columns:
    df_games['id'] = df_games['id'].astype(str)

    # Inicializar columnas si no existen
    if 'detailed_description' not in df_games.columns: df_games['detailed_description'] = ''
    if 'short_description' not in df_games.columns: df_games['short_description'] = ''

    # Carga el progreso de obtencion de la descripción
    if os.path.exists(ARCHIVO_SALIDA):
        print(f"\n🔄 Detectado archivo de progreso previo: {ARCHIVO_SALIDA}")
        print("   Intentando recuperar descripciones ya descargadas...")
        try:
            # Cargamos el CSV guardado
            df_progress = pd.read_csv(ARCHIVO_SALIDA, dtype={'id': str})

            # Mantenemos solo las columnas relevantes para evitar duplicados en merge
            cols_progreso = ['id', 'detailed_description', 'short_description']
            df_progress = df_progress[[c for c in cols_progreso if c in df_progress.columns]].dropna(subset=['id'])

            # Eliminamos duplicados en el progreso
            df_progress = df_progress.drop_duplicates(subset=['id'])

            # Hacemos merge con el dataframe principal
            # Usamos suffixes para distinguir lo nuevo de lo viejo
            df_games = df_games.merge(df_progress, on='id', how='left', suffixes=('', '_saved'))

            # Llenamos los valores vacios del original con los del archivo guardado
            df_games['detailed_description'] = df_games['detailed_description'].replace('', pd.NA)
            df_games['short_description'] = df_games['short_description'].replace('', pd.NA)

            df_games['detailed_description'] = df_games['detailed_description'].fillna(df_games['detailed_description_saved'])
            df_games['short_description'] = df_games['short_description'].fillna(df_games['short_description_saved'])

            # Limpiamos columnas temporales
            df_games.drop(columns=['detailed_description_saved', 'short_description_saved'], inplace=True, errors='ignore')

            descargados = df_games['detailed_description'].notna().sum()
            vacios = df_games['detailed_description'].isna().sum()
            print(f"Progreso recuperado: {descargados} juegos ya tienen descripción. Faltan {vacios}.-> {(df_games['detailed_description'] == '').sum()}")

        except Exception as e:
            print(f"Error al leer archivo de progreso: {e}. Se empezará desde cero.")


    # Consulta al api de Steam
    print(f"\n🌍 Iniciando/Continuando enriquecimiento con API Steam...")
    print(f"   Se guardará el progreso en '{ARCHIVO_SALIDA}' cada {GUARDAR_CADA_N} juegos.")

    total_games = len(df_games)
    start_time = time.time()

    df_games['detailed_description'] = df_games['detailed_description'].fillna('')
    df_games['short_description'] = df_games['short_description'].fillna('')

    for idx, row in df_games.iterrows():
        # Si ya cuenta con descripcion se salta al siguiente
        current_desc = str(row['detailed_description'])
        if len(current_desc) > 10:
            continue

        app_id = row['id']
        details = get_steam_game_details(app_id)

        if details:
            df_games.at[idx, 'detailed_description'] = details.get('detailed_description', '')
            df_games.at[idx, 'short_description'] = details.get('short_description', '')

        # Rate Limit: se espera 1.2s para evitar que el api nos bloquee por exeso de consultas.
        time.sleep(1.2)

        if idx % GUARDAR_CADA_N == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(f"   💾 [Auto-Save] Procesados {idx}/{total_games} | Tiempo: {elapsed:.0f}s")
            # Aplica la limpieza a las columnas de texto
            df_games['detailed_description'] = df_games['detailed_description'].apply(limpiar_texto)
            df_games['short_description'] = df_games['short_description'].apply(limpiar_texto)
            df_games.to_csv(ARCHIVO_SALIDA, index=False)

    print(f"💾 Guardado FINAL en {ARCHIVO_SALIDA}...")
    df_games.to_csv(ARCHIVO_SALIDA, index=False)


🔄 Detectado archivo de progreso previo: steam_games_enriched.csv
   Intentando recuperar descripciones ya descargadas...
✅ Progreso recuperado: 13825 juegos ya tienen descripción. Faltan 18310.-> 0

🌍 Iniciando/Continuando enriquecimiento con API Steam...
   Se guardará el progreso en 'steam_games_enriched.csv' cada 50 juegos.
   💾 [Auto-Save] Procesados 4600/32135 | Tiempo: 305s
   💾 [Auto-Save] Procesados 6150/32135 | Tiempo: 464s
   💾 [Auto-Save] Procesados 6650/32135 | Tiempo: 555s
   💾 [Auto-Save] Procesados 7300/32135 | Tiempo: 634s
   💾 [Auto-Save] Procesados 7700/32135 | Tiempo: 710s
   💾 [Auto-Save] Procesados 7850/32135 | Tiempo: 727s
   💾 [Auto-Save] Procesados 8000/32135 | Tiempo: 758s
   💾 [Auto-Save] Procesados 8050/32135 | Tiempo: 771s
   💾 [Auto-Save] Procesados 8400/32135 | Tiempo: 798s
   💾 [Auto-Save] Procesados 8950/32135 | Tiempo: 872s
   💾 [Auto-Save] Procesados 9200/32135 | Tiempo: 899s
   💾 [Auto-Save] Procesados 9850/32135 | Tiempo: 984s
   💾 [Auto-Save] Proce

In [ ]:
import pandas as pd

# Validamos que dataset se guardo correctamente.
df_games = pd.read_csv("steam_games_enriched.csv", dtype={'id': str})

In [ ]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore,detailed_description,short_description
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.49,http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140,Kotoshiro,NaN,NaN,Monsters appearing one after another. Let's su...,The monsters that are coming up one after anot...
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN,"<span class=""bb_img_ctn""><img class=""bb_img"" s...",Ironbound is a turn-based multiplayer strategy...
2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN,"<p class=""bb_paragraph"" >Real Pool 3D (Poolian...",Real Pool 3D (Poolians) is one of the best onl...
3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",0.83,http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,False,767400,彼岸领域,NaN,NaN,"The author of the first game, a bit special，<b...","The author of the first game, a bit special，Th..."
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"['Action', 'Indie', 'Casual', 'Sports']",1.79,http://steamcommunity.com/app/773570/reviews/?...,"['Single-player', 'Full controller support', '...",2.99,False,773570,NaN,NaN,NaN,Log Challenge is an arcade game where you will...,Log Challenge is an arcade game where you will...
